<a href="https://colab.research.google.com/github/lechurnair/TempQA/blob/main/ques_valid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

def check_temporal_connotation(predicted_question):
    doc = nlp(predicted_question)
    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
    return False

data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")

data["temporal_connotation"] = data["predicted_question"].apply(check_temporal_connotation)

# print(data)

num_true = data["temporal_connotation"].sum()

print("Number of predicted questions with temporal connotation:", num_true)

false_data = data[data["temporal_connotation"] == False]

print(false_data)

In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

def check_temporal_connotation(predicted_question):
    doc = nlp(predicted_question)
    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True
    return False

data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")

data["temporal_connotation"] = data["predicted_question"].apply(check_temporal_connotation)

num_true = data["temporal_connotation"].sum()

print("Number of predicted questions with temporal connotation:", num_true)

false_data = data[data["temporal_connotation"] == False]

print(false_data)


Number of predicted questions with temporal connotation: 95
Empty DataFrame
Columns: [context, real_question, predicted_question, temporal_connotation]
Index: []


In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c+=1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    words = []
    for token in context_doc:
        if token.pos_ in ["NOUN", "PROPN", "VERB"] and token.dep_ != "aux":
            words.append(token.lemma_.lower())

    # print(words)
    flag = False
    for word in words:
        if word and (word in [token.lemma_.lower() for token in doc]):
            counter += 1
            # print(word)
            # print('here')
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False


# # data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")
# data = pd.read_csv("/content/drive/MyDrive/Aru/question_cnnDailyMail.csv")
data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv",nrows=200)

# data["temporal_connotation"] = data.apply(lambda row: check_temporal_connotation(row["predicted_question"], row["context"]), axis=1)

# print(counter)
for index, row in data.iterrows():
    print("Question:", row["predicted_question"])
    result = check_temporal_connotation(row["predicted_question"], row["context"])
    print("Temporal Connotation:", result)
    print("-" * 50)
# true_data = data[data["temporal_connotation"] == True]

# print("Number of predicted questions with temporal connotation", len(true_data))
# true_data.to_csv("/content/drive/MyDrive/Aru/questions_with_temporal_connotation_cnn.csv", index=False)

# print(true_data)
# false_data = data[data["temporal_connotation"] == False]
# print("Number of predicted questions with temporal connotation", len(false_data))

# context = "[Typical Time] Ratners's chairman, Gerald Ratner, said the deal remains of \"substantial benefit to Ratners.\""
# context = "[Event Duration] When you inhale, air enters the respiratory system through your nose and ends up in your lungs, where gas exchange with the blood happens."
# context="[Frequency] At NEADS, the report of the hijacking was relayed immediately to Battle Commander Colonel Robert Marr."
# # context="[Frequency] Estimates of his date of birth vary by well over a century."
# check_temporal_connotation('How often did the hijacking take place?', context)

Question: How often does she need to contact Larry?
Temporal Connotation: True
--------------------------------------------------
Question: How long does it take to get the number?
Temporal Connotation: True
--------------------------------------------------
Question: When did she need to contact Larry?
Temporal Connotation: True
--------------------------------------------------
Question: What happens after she needs to contact Larry?
Temporal Connotation: True
--------------------------------------------------
Question: Is she still in need of the number?
Temporal Connotation: False
--------------------------------------------------
Question: How often do they watch a stand-up?
Temporal Connotation: True
--------------------------------------------------
Question: How long will the stand-up last?
Temporal Connotation: True
--------------------------------------------------
Question: When did they watch the stand-up?
Temporal Connotation: True
-----------------------------------------

In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c += 1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    noun_phrases = [chunk.text.lower() for chunk in context_doc.noun_chunks]

    flag = False
    for phrase in noun_phrases:
        phrase_doc = nlp(phrase)
        similarity_scores = [phrase_doc.similarity(token) for token in doc]
        max_score = max(similarity_scores, default=0)
        if max_score > 0.7:
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False


# data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")
# data = pd.read_csv("/content/drive/MyDrive/Aru/question_cnnDailyMail.csv")
data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv",nrows=200)
for index, row in data.iterrows():
    print("Question:", row["predicted_question"])
    result = check_temporal_connotation(row["predicted_question"], row["context"])
    print("Temporal Connotation:", result)
    print("-" * 50)
# data["temporal_connotation"] = data.apply(lambda row: check_temporal_connotation(row["predicted_question"], row["context"]), axis=1)

# # print(counter)

# true_data = data[data["temporal_connotation"] == True]

# print("Number of predicted questions with temporal connotation", len(true_data))

# false_data = data[data["temporal_connotation"] == False]

# print("Number of predicted questions with temporal connotation", len(false_data))
# # print(true_data)

# context = "[Typical Time] Ratners's chairman, Gerald Ratner, said the deal remains of \"substantial benefit to Ratners.\""
# context = "[Event Duration] When you inhale, air enters the respiratory system through your nose and ends up in your lungs, where gas exchange with the blood happens."
# context="[Frequency] At NEADS, the report of the hijacking was relayed immediately to Battle Commander Colonel Robert Marr."
# context="[Frequency] Estimates of his date of birth vary by well over a century."
# check_temporal_connotation('How often does he go to school?', context)

Question: How often does she need to contact Larry?
Temporal Connotation: True
--------------------------------------------------
Question: How long does it take to get the number?
Temporal Connotation: True
--------------------------------------------------
Question: When did she need to contact Larry?
Temporal Connotation: True
--------------------------------------------------
Question: What happens after she needs to contact Larry?
Temporal Connotation: True
--------------------------------------------------
Question: Is she still in need of the number?


<ipython-input-2-c4dd4e221f8f>:21: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [phrase_doc.similarity(token) for token in doc]


Temporal Connotation: False
--------------------------------------------------
Question: How often do they watch a stand-up?
Temporal Connotation: False
--------------------------------------------------
Question: How long will the stand-up last?
Temporal Connotation: False
--------------------------------------------------
Question: When did they watch the stand-up?
Temporal Connotation: False
--------------------------------------------------
Question: What happens after they watch the stand-up?
Temporal Connotation: False
--------------------------------------------------
Question: Is Eric and Rob still watching the stand-up?
Temporal Connotation: False
--------------------------------------------------
Question: How often does Lennny buy the trousers?
Temporal Connotation: True
--------------------------------------------------
Question: How long does Lennny's advice take?
Temporal Connotation: False
--------------------------------------------------
Question: When did Lennny buy t

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c += 1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    noun_phrases = [chunk.text.lower() for chunk in context_doc.noun_chunks]

    flag = False
    for phrase in noun_phrases:
        phrase_doc = nlp(phrase)
        similarity_scores = [phrase_doc.similarity(token) for token in doc]
        max_score = max(similarity_scores, default=0)
        if max_score > 0.6:
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False


# data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")
# data = pd.read_csv("/content/drive/MyDrive/Aru/question_cnnDailyMail.csv")
data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv")

data["temporal_connotation"] = data.apply(lambda row: check_temporal_connotation(row["predicted_question"], row["context"]), axis=1)

# print(counter)

true_data = data[data["temporal_connotation"] == True]

print("Number of predicted questions with temporal connotation", len(true_data))

false_data = data[data["temporal_connotation"] == False]

print("Number of predicted questions with temporal connotation", len(false_data))
# print(true_data)

# context = "[Typical Time] Ratners's chairman, Gerald Ratner, said the deal remains of \"substantial benefit to Ratners.\""
# context = "[Event Duration] When you inhale, air enters the respiratory system through your nose and ends up in your lungs, where gas exchange with the blood happens."
# context="[Frequency] At NEADS, the report of the hijacking was relayed immediately to Battle Commander Colonel Robert Marr."
# context="[Frequency] Estimates of his date of birth vary by well over a century."
# check_temporal_connotation('How often does he go to school?', context)

<ipython-input-2-90384800db68>:21: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [phrase_doc.similarity(token) for token in doc]


Number of predicted questions with temporal connotation 1599
Number of predicted questions with temporal connotation 2496


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/BTP Crowdsourcing (Responses) - Form Responses 1.csv")
ans=0
i=0
tp=0
tn=0
fp=0
fn=0
maj=0
for column in df.columns[2:]:
    value_counts = df[column].value_counts()

    majority_value = value_counts.idxmax()

    # print(f"Majority answer for {column}: {majority_value}")
    #print(majority_value)
    if majority_value=="I don't know":
      maj+=1
    elif majority_value=='Yes' and checklist[i]=='Yes':
      tp+=1
    elif majority_value=='No' and checklist[i]=='No':
      tn+=1
    elif majority_value=='Yes' and checklist[i]=='No':
      fn+=1
    elif majority_value=='No' and checklist[i]=='Yes':
      fp+=1
    if majority_value==checklist[i] or majority_value=="I don't know":
      ans+=1
    i+=1
print(ans)
df = pd.read_csv("/content/drive/MyDrive/Form2 - Form Responses 1.csv")
ans2=0
for column in df.columns[2:]:
    value_counts = df[column].value_counts()

    majority_value = value_counts.idxmax()

    # print(f"Majority answer for {column}: {majority_value}")
    #print(majority_value)
    if majority_value=="I don't know":
      if checklist[i]=='Yes':
        tp+=1
      else:
        tn+=1
    elif majority_value=='Yes' and checklist[i]=='Yes':
      tp+=1
    elif majority_value=='No' and checklist[i]=='No':
      tn+=1
    elif majority_value=='Yes' and checklist[i]=='No':
      fn+=1
    elif majority_value=='No' and checklist[i]=='Yes':
      fp+=1
    if majority_value==checklist[i] or majority_value=="I don't know":
      ans2+=1
    i+=1
print(ans2)
print(tp)
print(tn)
print(fp)
print(fn)
print(maj)

71
87
149
7
17
25
2


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/BTP Crowdsourcing (Responses) - Form Responses 1.csv")
ans=0
i=0
tp=0
tn=0
fp=0
fn=0
maj=0
for column in df.columns[2:]:
    value_counts = df[column].value_counts()

    majority_value = value_counts.idxmax()
    if majority_value=="I don't know":
      maj+=1
    elif majority_value=='Yes' and checklist_semantic[i]=='Yes':
      tp+=1
    elif majority_value=='No' and checklist_semantic[i]=='No':
      tn+=1
    elif majority_value=='Yes' and checklist_semantic[i]=='No':
      fn+=1
    elif majority_value=='No' and checklist_semantic[i]=='Yes':
      fp+=1
    # print(f"Majority answer for {column}: {majority_value}")
    #print(majority_value)
    if majority_value==checklist_semantic[i] or majority_value=="I don't know":
      ans+=1
    i+=1
print(ans)
df = pd.read_csv("/content/drive/MyDrive/Form2 - Form Responses 1.csv")
ans2=0
for column in df.columns[2:]:
    value_counts = df[column].value_counts()

    majority_value = value_counts.idxmax()
    if majority_value=="I don't know":
      if checklist_semantic[i]=='Yes':
        tp+=1
      else:
        tn+=1
    elif majority_value=='Yes' and checklist_semantic[i]=='Yes':
      tp+=1
    elif majority_value=='No' and checklist_semantic[i]=='No':
      tn+=1
    elif majority_value=='Yes' and checklist_semantic[i]=='No':
      fn+=1
    elif majority_value=='No' and checklist_semantic[i]=='Yes':
      fp+=1
    # print(f"Majority answer for {column}: {majority_value}")
    #print(majority_value)
    if majority_value==checklist_semantic[i] or majority_value=="I don't know":
      ans2+=1
    i+=1
print(ans2)
print(tp)
print(tn)
print(fp)
print(fn)
print(maj)

71
65
121
13
11
53
2


In [ ]:
print(i)

200


In [ ]:
#lexical
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c+=1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    words = []
    for token in context_doc:
        if token.pos_ in ["NOUN", "PROPN", "VERB"] and token.dep_ != "aux":
            words.append(token.lemma_.lower())

    # print(words)
    flag = False
    for word in words:
        if word and (word in [token.lemma_.lower() for token in doc]):
            counter += 1
            # print(word)
            # print('here')
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False


# # data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")
# data = pd.read_csv("/content/drive/MyDrive/Aru/question_cnnDailyMail.csv")
data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv",nrows=200)

# data["temporal_connotation"] = data.apply(lambda row: check_temporal_connotation(row["predicted_question"], row["context"]), axis=1)

# print(counter)
checklist=[]
for index, row in data.iterrows():
    # print("Question:", row["predicted_question"])
    result = check_temporal_connotation(row["predicted_question"], row["context"])
    if result:
      # print('Yes')
      checklist.append('Yes')
    else:
      # print('No')
      checklist.append('No')
    # print("Temporal Connotation:", result)
    # print("-" * 50)
print(len(checklist))
# true_data = data[data["temporal_connotation"] == True]

# print("Number of predicted questions with temporal connotation", len(true_data))
# true_data.to_csv("/content/drive/MyDrive/Aru/questions_with_temporal_connotation_cnn.csv", index=False)

# print(true_data)
# false_data = data[data["temporal_connotation"] == False]
# print("Number of predicted questions with temporal connotation", len(false_data))

# context = "[Typical Time] Ratners's chairman, Gerald Ratner, said the deal remains of \"substantial benefit to Ratners.\""
# context = "[Event Duration] When you inhale, air enters the respiratory system through your nose and ends up in your lungs, where gas exchange with the blood happens."
# context="[Frequency] At NEADS, the report of the hijacking was relayed immediately to Battle Commander Colonel Robert Marr."
# # context="[Frequency] Estimates of his date of birth vary by well over a century."
# check_temporal_connotation('How often did the hijacking take place?', context)

200


In [ ]:
# old semantic
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c += 1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    noun_phrases = [chunk.text.lower() for chunk in context_doc.noun_chunks]

    flag = False
    for phrase in noun_phrases:
        phrase_doc = nlp(phrase)
        similarity_scores = [phrase_doc.similarity(token) for token in doc]
        max_score = max(similarity_scores, default=0)
        if max_score > 0.5:
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False


# data = pd.read_excel("/content/drive/MyDrive/generated_questions_sampling.xlsx")
# data = pd.read_csv("/content/drive/MyDrive/Aru/question_cnnDailyMail.csv")
data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv",nrows=200)
checklist_semantic=[]
for index, row in data.iterrows():
    # print("Question:", row["predicted_question"])
    result = check_temporal_connotation(row["predicted_question"], row["context"])
    if result:
      # print('Yes')
      checklist_semantic.append('Yes')
    else:
      # print('No')
      checklist_semantic.append('No')
    # print("Temporal Connotation:", result)
    # print("-" * 50)
print(len(checklist_semantic))
# data["temporal_connotation"] = data.apply(lambda row: check_temporal_connotation(row["predicted_question"], row["context"]), axis=1)

# # print(counter)

# true_data = data[data["temporal_connotation"] == True]

# print("Number of predicted questions with temporal connotation", len(true_data))

# false_data = data[data["temporal_connotation"] == False]

# print("Number of predicted questions with temporal connotation", len(false_data))
# # print(true_data)

# context = "[Typical Time] Ratners's chairman, Gerald Ratner, said the deal remains of \"substantial benefit to Ratners.\""
# context = "[Event Duration] When you inhale, air enters the respiratory system through your nose and ends up in your lungs, where gas exchange with the blood happens."
# context="[Frequency] At NEADS, the report of the hijacking was relayed immediately to Battle Commander Colonel Robert Marr."
# context="[Frequency] Estimates of his date of birth vary by well over a century."
# check_temporal_connotation('How often does he go to school?', context)

<ipython-input-5-71fe8cfa6d35>:22: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [phrase_doc.similarity(token) for token in doc]


200


In [ ]:
# old semantic
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c += 1
    doc = nlp(predicted_question)
    print(doc)
    context_doc = nlp(context)
    noun_phrases = [chunk.text.lower() for chunk in context_doc.noun_chunks]
    print(noun_phrases)

    flag = False
    for phrase in noun_phrases:
        phrase_doc = nlp(phrase)
        similarity_scores = [phrase_doc.similarity(token) for token in doc]
        if phrase=='the store':
          print(phrase_doc)
          print(similarity_scores)
        max_score = max(similarity_scores, default=0)
        if max_score > 0.5:
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False

ques= 'When did John go to the store?'
context = "John went to the store yesterday to buy groceries. He met his friend there, and they had lunch together."
context="Eric and Rob are going to watch a stand-up on youtube"
ques='How long is the stand-up'
check_temporal_connotation(ques, context)

When did John go to the store?
['john', 'the store', 'groceries', 'he', 'his friend', 'they', 'lunch']
the store
[0.11017632374642286, -0.10258677771377366, 0.2671390988119976, -0.01342444363386507, 0.035804172959673676, 0.4834658575775949, 0.440631212656286, 0.06616876911514795]


<ipython-input-11-90d27c1a5dae>:23: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [phrase_doc.similarity(token) for token in doc]


False

In [ ]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter = 0
c = 0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c += 1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    phrases = [str(chunk) for chunk in context_doc.noun_chunks] + [str(chunk) for chunk in context_doc.sents]
    print(phrases)

    flag = False
    for phrase in phrases:
        phrase_doc = nlp(phrase)
        similarity_scores = [phrase_doc.similarity(token) for token in doc]
        max_score = max(similarity_scores, default=0)
        if max_score > 0.5:
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False

# ques= 'When did John go to the store?'
ques='Is she still in need of the number?'
# context = "John went to the store yesterday to buy groceries. He met his friend there, and they had lunch together."
context="Hannah needs the number but Amanda doesn't have it. She needs to contact Larry."
context="Eric and Rob are going to watch stand-up by XYZ"
ques='How long is the stand-up'
check_temporal_connotation(ques, context)

['Eric', 'Rob', 'stand-up', 'XYZ', 'Eric and Rob are going to watch stand-up by XYZ']


<ipython-input-5-29998b601ac0>:22: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [phrase_doc.similarity(token) for token in doc]


True

In [ ]:
#lexical
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

counter=0
c=0

def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c+=1
    doc = nlp(predicted_question)

    context_doc = nlp(context)
    words = []
    for token in context_doc:
        if token.pos_ in ["NOUN", "PROPN", "VERB"] and token.dep_ != "aux":
            words.append(token.lemma_.lower())

    # print(words)
    flag = False
    for word in words:
        if word and (word in [token.lemma_.lower() for token in doc]):
            counter += 1
            # print(word)
            # print('here')
            flag = True
            break

    if not flag:
        return False
    print('here')
    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False
ques='Will she still need the number?'
# context = "John went to the store yesterday to buy groceries. He met his friend there, and they had lunch together."
context="Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."
context="Eric and Rob are going to watch Roy's stand-up on youtube"
ques='How long is the stand-up'
check_temporal_connotation(ques, context)

here


True

In [ ]:
#latest semantic
counter=0
c=0
def check_temporal_connotation(predicted_question, context):
    global counter
    global c
    c += 1
    doc = nlp(predicted_question)
    context_doc = nlp(context)
    noun_phrases = [chunk.text.lower() for chunk in context_doc.noun_chunks]

    words = [token.text.lower() for token in doc if token.pos_ in ["NOUN", "PROPN", "VERB"]]
    phrases = [' '.join(words[i:i+2]) for i in range(len(words) - 1)]

    flag = False
    for phrase in phrases:
        phrase_doc = nlp(phrase)
        similarity_scores = [phrase_doc.similarity(nlp(np)) for np in noun_phrases]
        max_score = max(similarity_scores, default=0)
        if max_score > 0.5:
            counter += 1
            flag = True
            break

    if not flag:
        return False

    for token in doc:
        if token.dep_ == "advmod":
            if token.text.lower() in ["how", "when", "how often", "how long", "how frequent"]:
                return True
        elif token.dep_ == "aux" and token.text.lower() == "when":
            return True
        elif token.dep_ == "prep" and token.text.lower() in ["during", "at"]:
            return True
        elif token.text.lower() in ["after", "before", "tomorrow", "yesterday"]:
            return True
        elif token.dep_ == "aux" and token.text.lower() == "will":
            return True

    return False


data = pd.read_csv("/content/drive/MyDrive/Aru/question_Samsum_test.csv",nrows=200)
checklist_semantic=[]
for index, row in data.iterrows():
    # print("Question:", row["predicted_question"])
    result = check_temporal_connotation(row["predicted_question"], row["context"])
    if result:
      # print('Yes')
      checklist_semantic.append('Yes')
    else:
      # print('No')
      checklist_semantic.append('No')
    # print("Temporal Connotation:", result)
    # print("-" * 50)
print(len(checklist_semantic))

<ipython-input-24-bd21e4b28d8b>:19: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_scores = [phrase_doc.similarity(nlp(np)) for np in noun_phrases]


200


In [ ]:
import pandas as pd

input_file = "/content/drive/MyDrive/generated_questions_sampling.xlsx"
output_file = "/content/drive/MyDrive/Aru/outputnew.xlsx"

df = pd.read_excel(input_file)

for index, row in df.iterrows():
    if row['context'] == "Event Ordering" and ("after" in row['real_question'] or "after" in row['predicted_question']):
        df.at[index, 'real_question'] = row['real_question'].replace('after', 'before')
        df.at[index, 'predicted_question'] = row['predicted_question'].replace('after', 'before')

df.to_excel(output_file, index=False)

print("Replacement completed. Output saved to", output_file)


Replacement completed. Output saved to /content/drive/MyDrive/Aru/outputnew.xlsx


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Question Answering (Responses) - Form Responses 1.csv')

df = df.iloc[:, 2:]

selected_rows = df.iloc[[0, 2, 3]]

count_yes = 0
count_no = 0
count_idk = 0
count_equal = 0
count_how = 0
count_how_often=0
count_when = 0
count_what = 0
count_is = 0

def extract_question(full_string):
    parts = full_string.split("Answer:")
    if len(parts) > 1:
        question = parts[0].replace('Question:', '').strip()
        return question
    return None

for col in selected_rows.columns:
    question = extract_question(col)
    if question.startswith("How long"):
        count_how+=1
    elif question.startswith("How often") or question.startswith("How many"):
        count_how_often+=1
    elif question.startswith("When"):
        count_when+=1
    elif question.startswith("What"):
        count_what+=1
    elif question.startswith("Is"):
            count_is+=1
    values = selected_rows[col].value_counts()
    if 'Yes' not in values:
        values['Yes'] = 0
    if 'No' not in values:
        values['No'] = 0
    if "I don't know" not in values:
        values["I don't know"] = 0

    yes_count = values['Yes']
    no_count = values['No']
    idk_count = values["I don't know"]

    if yes_count > no_count and yes_count > idk_count:
        count_yes += 1
    elif no_count > yes_count and no_count > idk_count:
        count_no += 1
    elif idk_count > yes_count and idk_count > no_count:
        count_idk += 1
    else:
        count_equal += 1

print(f"Yes: {count_yes}")
print(f"No: {count_no}")
print(f"I don't know: {count_idk}")
print(f"Equal count: {count_equal}")


Yes: 73
No: 15
I don't know: 4
Equal count: 8


In [ ]:
import pandas as pd
import re

df = pd.read_csv('/content/drive/MyDrive/Question Answering (Responses) - Form Responses 1.csv')

df = df.iloc[:, 2:]

selected_rows = df.iloc[[0, 2, 3]]

count_yes = 0
count_no = 0
count_idk = 0
count_equal = 0
count_how = 0
count_how_often=0
count_when = 0
count_what = 0
count_is = 0

def extract_question(full_string):
    question_index = full_string.find("Question:")
    answer_index = full_string.find("Answer:")

    if question_index != -1 and answer_index != -1:
        question = full_string[question_index + len("Question:"):answer_index].strip()
        return question
    return None

for col in selected_rows.columns:
    question = extract_question(col)
    # print(question)
    if question.startswith("How often") or question.startswith("How many"):
        count_how+=1
        values = selected_rows[col].value_counts()
        if 'Yes' not in values:
            values['Yes'] = 0
        if 'No' not in values:
            values['No'] = 0
        if "I don't know" not in values:
            values["I don't know"] = 0

        yes_count = values['Yes']
        no_count = values['No']
        idk_count = values["I don't know"]

        if yes_count > no_count and yes_count > idk_count:
            count_yes += 1
        elif no_count > yes_count and no_count > idk_count:
            count_no += 1
        elif idk_count > yes_count and idk_count > no_count:
            count_idk += 1
        else:
            count_equal += 1
    elif question.startswith("How often") or question.startswith("How many"):
        count_how_often+=1
    elif question.startswith("When"):
        count_when+=1
    elif question.startswith("What"):
        count_what+=1
    elif question.startswith("Is"):
        count_is+=1

print(f"Yes: {count_yes}")
print(f"No: {count_no}")
print(f"I don't know: {count_idk}")
print(f"Equal count: {count_equal}")
print(count_how)
print(count_how_often)
print(count_when)
print(count_what)
print(count_is)

Yes: 19
No: 7
I don't know: 0
Equal count: 3
29
0
30
9
2
